In [ ]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
# Load CSV file (exported from InfluxDB)
df = pd.read_csv("./data/battery_data.csv")

# Convert timestamp to datetime for time-series processing
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Simulate Time-to-Failure (assuming failure happens at the last recorded timestamp)
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600  # Convert to hours

# Display processed data
print(df.head())

In [ ]:
# Select Features & Target Variable
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Regression Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Make Predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.2f} hours before failure")

In [ ]:
# Save the trained model
joblib.dump(model, "./models/battery_ttf_model.pkl")
print("Model saved as battery_ttf_model.pkl")